# 🌍 Carbon Emission Prediction Model
A machine learning model to predict CO₂ emissions using historical data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## 📥 Load and Prepare Dataset

In [ ]:
# Load dataset
df = pd.read_csv("f90871f6-e095-47ea-8d68-4aff4b0bf44a.csv")

# Reshape and clean data
df_long = df.melt(
    id_vars=["Country code", "Country name", "Series code", "Series name", "SCALE", "Decimals"],
    var_name="Year",
    value_name="Value"
)
df_long["Value"] = df_long["Value"].replace("..", pd.NA)
df_long.dropna(subset=["Value"], inplace=True)
df_long["Value"] = df_long["Value"].astype(float)
df_long["Year"] = df_long["Year"].astype(int)

## 🧠 Encode Categorical Variables

In [ ]:
# Label Encoding
le_country = LabelEncoder()
le_series = LabelEncoder()
df_long["Country_encoded"] = le_country.fit_transform(df_long["Country name"])
df_long["Series_encoded"] = le_series.fit_transform(df_long["Series name"])

## ✨ Train the Random Forest Regressor

In [ ]:
# Define features and target
X = df_long[["Country_encoded", "Series_encoded", "Year"]]
y = df_long["Value"]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

## 🔮 Make Sample Predictions

In [ ]:
# Sample predictions
X_test_sample = X_test.iloc[:5]
y_pred_sample = model.predict(X_test_sample)

# Decode results for readability
output_sample = X_test_sample.copy()
output_sample["Country"] = le_country.inverse_transform(output_sample["Country_encoded"])
output_sample["Series"] = le_series.inverse_transform(output_sample["Series_encoded"])
output_sample["Predicted CO₂ Emission"] = np.round(y_pred_sample, 2)
output_sample = output_sample[["Country", "Year", "Series", "Predicted CO₂ Emission"]]
output_sample.reset_index(drop=True, inplace=True)
output_sample